## <b>San Antonio Universities for Students Who don’t Own a Vehicle</b>
### Adam Andre, April 10, 2020

<b>1. Introduction</b>

Which university in the San Antonio area have the most amenities within walking distance? I would have to leverage the Foursquare location data in a small radius around each campus. Answering this question can help potential students (stakeholders) who are planning on living on campus who do not have a vehicle choose a school.

<b>2. Data</b>

I will be utilizing a data set I put together containing eight San Antonio universities, their latitudes and longitudes, and their address. I will use this data set in combination with the Foursquare API to determine how many restaurants, grocery stores, parks, shops, and bus stations are within walking distance of each university. I will determine a weighted score for each category and determine which university is best for students living on campus without a vehicle.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


<b>The dataframe containing the data on the universities</b>

In [2]:
df_uni = pd.read_csv('SATXUniversities.csv')
df_uni

,University,Latitude,Longitude,Address
0,The University of Texas at San Antonio,29.584057,-98.626059,"1 UTSA Circle, San Antonio, TX 78249"
1,Trinity University,29.463490,-98.481873,"1 Trinity Pl, San Antonio, TX 78212"
2,Texas Lutheran University,29.569571,-97.984154,"1000 W Court St, Seguin, TX 78155"
3,St. Mary's University,29.453159,-98.562637,"1 Camino Santa Maria, San Antonio, TX 78228"
4,Our Lady of the Lake University,29.426729,-98.544159,"411 SW 24th St, San Antonio, TX 78207"
5,University of the Incarnate Word,29.470289,-98.472252,"4301 Broadway St, San Antonio, TX 78209"
6,Texas A&M University-San Antonio,29.312670,-98.524811,"One University Way, San Antonio, TX 78224"
7,Texas State University,29.889000,-97.934547,"601 University Dr, San Marcos, TX 78666"


<b>A map of the universities</b>

In [3]:
#Map of the universities
latitude = 29.555071
longitude = -98.263519

uni_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng in zip(df_uni.Latitude, df_uni.Longitude):
    folium.features.Marker(
        location=[lat, lng]
    ).add_to(uni_map)
    
uni_map

<b>Define Foursquare credentials</b>

In [4]:
CLIENT_ID = 'ZZYDWMSYKT0GTK3YFKQQMTEPVMK3U3FLUEQCYZTEMLUVIZYL' # your Foursquare ID
CLIENT_SECRET = '0RMTNKUTQU0UV0C1HCXGDFGPLPGGFIZA2O3T3ASMHMEV0APD' # your Foursquare Secret
VERSION = '20200401' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZZYDWMSYKT0GTK3YFKQQMTEPVMK3U3FLUEQCYZTEMLUVIZYL
CLIENT_SECRET:0RMTNKUTQU0UV0C1HCXGDFGPLPGGFIZA2O3T3ASMHMEV0APD


<b>Prepare dataframe for counts of each amenity</b>

In [5]:
df_uni['Restaurants'] = 0
df_uni['Coffee shops'] = 0
df_uni['Parks and Gyms'] = 0
df_uni['Shops'] = 0
df_uni['Bars'] = 0
df_uni

,University,Latitude,Longitude,Address,Restaurants,Coffee shops,Parks and Gyms,Shops,Bars
0,The University of Texas at San Antonio,29.584057,-98.626059,"1 UTSA Circle, San Antonio, TX 78249",0,0,0,0,0
1,Trinity University,29.463490,-98.481873,"1 Trinity Pl, San Antonio, TX 78212",0,0,0,0,0
2,Texas Lutheran University,29.569571,-97.984154,"1000 W Court St, Seguin, TX 78155",0,0,0,0,0
3,St. Mary's University,29.453159,-98.562637,"1 Camino Santa Maria, San Antonio, TX 78228",0,0,0,0,0
4,Our Lady of the Lake University,29.426729,-98.544159,"411 SW 24th St, San Antonio, TX 78207",0,0,0,0,0
5,University of the Incarnate Word,29.470289,-98.472252,"4301 Broadway St, San Antonio, TX 78209",0,0,0,0,0
6,Texas A&M University-San Antonio,29.312670,-98.524811,"One University Way, San Antonio, TX 78224",0,0,0,0,0
7,Texas State University,29.889000,-97.934547,"601 University Dr, San Marcos, TX 78666",0,0,0,0,0


<b>Get count of each amenity for each university from Foursquare API</b>

In [6]:
radius = 600 #walking distance is generally agreed to be 400 meters (0.25 miles), but I am increasing to 600 for better data set
queries = [['Restaurants', 'food'], ['Coffee shops', 'coffee'], ['Parks and Gyms', 'outdoors'], ['Shops', 'shops'], ['Bars', 'drinks']]

for lat, lng, i in zip(df_uni.Latitude, df_uni.Longitude, df_uni.index):
    for query in queries:
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&section={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, query[1], radius)
        results = requests.get(url).json()
        venues = results['response']['groups'][0]['items']
        df_venues = json_normalize(venues)
        df_uni.loc[df_uni.index[i], query[0]] = len(df_venues.index)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


In [7]:
df_uni

,University,Latitude,Longitude,Address,Restaurants,Coffee shops,Parks and Gyms,Shops,Bars
0,The University of Texas at San Antonio,29.584057,-98.626059,"1 UTSA Circle, San Antonio, TX 78249",7,2,5,7,1
1,Trinity University,29.463490,-98.481873,"1 Trinity Pl, San Antonio, TX 78212",4,2,4,1,0
2,Texas Lutheran University,29.569571,-97.984154,"1000 W Court St, Seguin, TX 78155",4,2,2,5,2
3,St. Mary's University,29.453159,-98.562637,"1 Camino Santa Maria, San Antonio, TX 78228",2,2,4,5,3
4,Our Lady of the Lake University,29.426729,-98.544159,"411 SW 24th St, San Antonio, TX 78207",7,1,6,17,1
5,University of the Incarnate Word,29.470289,-98.472252,"4301 Broadway St, San Antonio, TX 78209",1,2,6,2,0
6,Texas A&M University-San Antonio,29.312670,-98.524811,"One University Way, San Antonio, TX 78224",0,0,0,0,0
7,Texas State University,29.889000,-97.934547,"601 University Dr, San Marcos, TX 78666",4,2,14,5,3


In [16]:
df_uni_clean = df_uni.drop([6]).reset_index() #drop Texas A&M - no data
df_uni_clean.columns = [c.replace(' ', '_') for c in df_uni_clean.columns]
df_uni_clean

,index,University,Latitude,Longitude,Address,Restaurants,Coffee_shops,Parks_and_Gyms,Shops,Bars
0,0,The University of Texas at San Antonio,29.584057,-98.626059,"1 UTSA Circle, San Antonio, TX 78249",7,2,5,7,1
1,1,Trinity University,29.463490,-98.481873,"1 Trinity Pl, San Antonio, TX 78212",4,2,4,1,0
2,2,Texas Lutheran University,29.569571,-97.984154,"1000 W Court St, Seguin, TX 78155",4,2,2,5,2
3,3,St. Mary's University,29.453159,-98.562637,"1 Camino Santa Maria, San Antonio, TX 78228",2,2,4,5,3
4,4,Our Lady of the Lake University,29.426729,-98.544159,"411 SW 24th St, San Antonio, TX 78207",7,1,6,17,1
5,5,University of the Incarnate Word,29.470289,-98.472252,"4301 Broadway St, San Antonio, TX 78209",1,2,6,2,0
6,7,Texas State University,29.889000,-97.934547,"601 University Dr, San Marcos, TX 78666",4,2,14,5,3


<b>3. Methodology</b>

I will create a model to give each university a score between 0 and 100.

Some categories are worth more, so the model will assign scores as follows:<br>
Restaurants: 30%<br>
Coffee: 10%<br>
Parks: 30%<br>
Shops: 20%<br>
Bars: 10%<br>
Total: 100%

In [20]:
print('Scores for each university:')

for name, r, c, p, s, b in zip(df_uni_clean.University, df_uni_clean.Restaurants, df_uni_clean.Coffee_shops, df_uni_clean.Parks_and_Gyms, df_uni_clean.Shops, df_uni_clean.Bars):
    score = 0
    score += r/7 * 30
    score += c/2 * 10
    score += p/14 * 30
    score += s/17 * 20
    score += b/3 * 10
    print(name, ': ', score)

Scores for each university:
The University of Texas at San Antonio :  62.28291316526611
Trinity University :  36.89075630252101
Texas Lutheran University :  43.97759103641456
St. Mary's University :  43.02521008403361
Our Lady of the Lake University :  71.19047619047619
University of the Incarnate Word :  29.495798319327726
Texas State University :  73.0252100840336


Ranked:

1. Texas State University - 73<br>
2. Our Lady of the Lake University - 71<br>
3. The University of Texas at San Antonio - 62<br>
4. Texas Lutheran University - 44<br>
5. St. Mary's University - 43<br>
6. Trinity University - 37<br>
7. University of the Incarnate Word - 29